# Computate Smart Cloud Builder

## About the open source GPL3 license and copyright for this product

Copyright © 2024 Computate Limited Liability Company in Utah, USA

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

ADDITIONAL TERMS

As stated in section 7. c) and e) of the GPL3 license, 
"you may supplement the terms of this License with terms," 
Computate has added the following additional terms to the license: 

  7 c) Prohibiting misrepresentation of the origin of that material, and
    requiring that modified versions of such material be marked in
    reasonable ways as different from the original version;

  7 e) Declining to grant rights under trademark law for use of some
    trade names, trademarks, or service marks;

Please do not redistribute this course until you have built your own platform with these tools, 
separate from the computate.org platform, and reconfigure your fork of this repo to deploy 
your own platform instead of the computate.org platform. 

QUESTIONS

For questions about this open source license, please contact our public mailing list at computate@group.computate.org


# Load the code search model into the Solr search engine

In this hackathon, we will be generating code with AI/ML provided by the well established [computate project](https://github.com/computate-org/computate). We will automatically generate OpenAPI specs, database table schemas, Java POJOs, Vert.x Reactive Java APIs, Handlebars HTML page templates, JavaScript page and API functions, NGSI-LD Context data, OpenShift Custom Resource Definitions, and Ansible Operator roles and playbooks for any of the hundreds of open source Edge device data related [FIWARE Smart Data Models available here](https://github.com/smart-data-models). 

## Prerequisites
To run the AI/ML code generation, you need to have completed the steps in the [README](README.md), [notebook 01-install-prerequisites.ipynb](01-install-prerequisites.ipynb), and [ notebook 02-deploy-microservices.ipynb](01-install-prerequisites.ipynb) to install the dependencies into the workbench, and deploy all the microservices. Then run the commands below in your OpenShift AI Workbench. 

## Clone Computate and Smart Village repos

### Install the computate project

The [computate project](https://github.com/computate-org/computate.git) is a Java project that watches for changes to files in a directory recursively, parses the Java code as it's created or updated, indexes every detail about each Java class, constructor, method, and field in the Apache Solr search engine, and generates code based on what it discovers. This is the main open source library that does the AI/ML code generation. Instead of a typical model server, we use Apache Solr as the model server. We can flexibly index every aspect about our Java code in Solr as we code over time, including multiple Java projects at the same time, and link together classes, types, and foreign key relations between projects. 

In [ ]:
eval $(./vars.py)
git clone https://github.com/computate-org/computate.git $COMPUTATE_SRC
echo DONE

Use maven to compile and install the computate project in the workbench. 

In [1]:
eval $(./vars.py)
(cd $COMPUTATE_SRC && mvn clean install)
echo DONE

[INFO] Scanning for projects...
[INFO] Inspecting build with total of 1 modules...
[INFO] Installing Nexus Staging features:
[INFO]   ... total of 1 executions of maven-deploy-plugin replaced with nexus-staging-maven-plugin
[INFO] 
[INFO] ----------------------< org.computate:computate >-----------------------
[INFO] Building Computate Vert.x Code Generation Tools 4.4.4.2
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-clean-plugin:2.5:clean (default-clean) @ computate ---
[INFO] Deleting /opt/app-root/src/computate/target
[INFO] 
[INFO] --- flatten-maven-plugin:1.0.0:clean (flatten.clean) @ computate ---
[INFO] Deleting /opt/app-root/src/computate/.flattened-pom.xml
[INFO] 
[INFO] --- build-helper-maven-plugin:3.0.0:add-source (default) @ computate ---
[INFO] Source directory: /opt/app-root/src/computate/src/gen/java added.
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ computate ---
[WARNING] Usin

### Install the computate-base project

The [computate-base project](https://github.com/computate-org/computate-base.git) contains the default Maven versions for all the dependencies. 

In [ ]:
eval $(./vars.py)
git clone https://github.com/computate-org/computate-base.git $COMPUTATE_SRC/../computate-base
echo DONE

Use maven to compile and install the computate project in the workbench. 

In [ ]:
eval $(./vars.py)
(cd $COMPUTATE_SRC/../computate-base && env mvn clean install)
echo DONE

### Install the computate-search project

The [computate-search project](https://github.com/computate-org/computate-search.git) is a Java project that mainly interacts with Apache Solr to make search queries, and parse the response. There are also several date time serializers and deserializers for handling date times in requests. One other important class is the `Wrap` class which can wrap a Java field initialization value by it's generic type as part of the generated initialization code for each class. 

In [ ]:
eval $(./vars.py)
git clone https://github.com/computate-org/computate-search.git $COMPUTATE_SEARCH_SRC
echo DONE

Run the Ansible Playbook to configure and compile the `computate-search` project. 
This will index all of the `computate-search` Java classes into the Apache Solr search engine. 

In [ ]:
eval $(./vars.py)
ansible-playbook -e @$VARS_PATH playbooks/computate-project.yaml \
  -e SYSTEMD_ENABLED=false \
  -e SITE_NAME=computate-search \
  -e SITE_PREFIX="$SITE_PREFIX" \
  -e SOLR_HOST_NAME="$SOLR_HOST_NAME" \
  -e SOLR_PORT="$SOLR_PORT" \
  -e SOLR_SSL="$SOLR_SSL" \
  -e SOLR_URL="$SOLR_URL_COMPUTATE" \
  -e SOLR_URL_COMPUTATE="$SOLR_URL_COMPUTATE"
echo DONE

Take a look at the `computate-search/config/computate-search.yaml` configuration file that was created based on your environment. 

In [ ]:
eval $(./vars.py)
cat $COMPUTATE_SEARCH_SRC/vars.yaml
echo DONE

Now query the Solr search engine to find out how many Java classes, constructors, methods, fields, and generated fields are found in the  `computate-search` project. 

In [ ]:
eval $(./vars.py)
curl -k -s -u "$SOLR_USERNAME:$SOLR_PASSWORD" "$SOLR_URL_COMPUTATE"'/query?rows=0&fq=siteNom_indexed_string:computate-search' -d \
  '{
    "query": "*:*"
    , "facet": {
      "classes" : { "type": "terms", "field": "classeNomSimple_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "constructors" : { "type": "terms", "field": "partEstConstructeur_indexed_boolean" }
      , "methods" : { "type": "terms", "field": "partEstMethode_indexed_boolean" }
      , "methodNames" : { "type": "terms", "field": "methodeVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "fields" : { "type": "terms", "field": "partEstChamp_indexed_boolean" }
      , "fieldNames" : { "type": "terms", "field": "champVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "generatedFields" : { "type": "terms", "field": "partEstEntite_indexed_boolean" }
      , "generatedFieldNames" : { "type": "terms", "field": "entiteVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
    }
  }' | jq -r '"\(.facets.classes.numBuckets) Java classes (for example \(.facets.classes.buckets | map(.val) | join(", ")))
\(.facets.constructors.buckets[0].count) constructors
\(.facets.methods.buckets[0].count) methods (for example \(.facets.methodNames.buckets | map(.val) | join(", ")))
\(.facets.fields.buckets[0].count) fields (for example \(.facets.fieldNames.buckets | map(.val) | join(", ")))
\(.facets.generatedFields.buckets[0].count) new generated fields (for example \(.facets.generatedFieldNames.buckets | map(.val) | join(", ")))"'
echo DONE

### Install the computate-vertx project

The [computate-vertx project](https://github.com/computate-org/computate-vertx.git) is a Java project that integrates Vert.x libraries into a project for reactive, event-bus driven software development. There is additional search capabilities built in to perform searches to Apache Solr asynchronously. There is PostgreSQL database integration with serializers and deserializers for GeoJson fields. Additional Vert.x tools, OpenAPI schema writers, boiler plate code project writers, FIWARE and NGSI-LD context writers, OpenID Connect/OAuth2 authentication/authorization handlers, Handlebars HTML Template handlers, and a base API Java Interface for building powerful, reactive secure APIs for anything. 

In [ ]:
eval $(./vars.py)
git clone https://github.com/computate-org/computate-vertx.git $COMPUTATE_VERTX_SRC
echo DONE

Run the Ansible Playbook to configure and compile the `computate-vertx` project. 
This will index all of the `computate-vertx` Java classes into the Apache Solr search engine. 

In [ ]:
eval $(./vars.py)
ansible-playbook -e @$VARS_PATH playbooks/computate-project.yaml \
  -e SYSTEMD_ENABLED=false \
  -e SITE_NAME=computate-vertx \
  -e SITE_PREFIX="$SITE_PREFIX" \
  -e SOLR_HOST_NAME="$SOLR_HOST_NAME" \
  -e SOLR_PORT="$SOLR_PORT" \
  -e SOLR_SSL="$SOLR_SSL" \
  -e SOLR_URL="$SOLR_URL_COMPUTATE" \
  -e SOLR_URL_COMPUTATE="$SOLR_URL_COMPUTATE"
echo DONE

Take a look at the `computate-vertx/config/computate-search.yaml` configuration file that was created based on your environment. 

In [ ]:
cat $COMPUTATE_VERTX_SRC/vars.yaml
echo DONE

Now query the Solr search engine to find out how many Java classes, constructors, methods, fields, and generated fields are found in the  `computate-vertx` project. 

In [ ]:
eval $(./vars.py)
curl -k -s -u "$SOLR_USERNAME:$SOLR_PASSWORD" "$SOLR_URL_COMPUTATE"'/query?rows=0&fq=siteNom_indexed_string:computate-vertx' -d \
  '{
    "query": "*:*"
    , "facet": {
      "classes" : { "type": "terms", "field": "classeNomSimple_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "constructors" : { "type": "terms", "field": "partEstConstructeur_indexed_boolean" }
      , "methods" : { "type": "terms", "field": "partEstMethode_indexed_boolean" }
      , "methodNames" : { "type": "terms", "field": "methodeVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "fields" : { "type": "terms", "field": "partEstChamp_indexed_boolean" }
      , "fieldNames" : { "type": "terms", "field": "champVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
      , "generatedFields" : { "type": "terms", "field": "partEstEntite_indexed_boolean" }
      , "generatedFieldNames" : { "type": "terms", "field": "entiteVar_enUS_indexed_string", "numBuckets": true, "limit": 5 }
    }
  }' | jq -r '"\(.facets.classes.numBuckets) Java classes (for example \(.facets.classes.buckets | map(.val) | join(", ")))
\(.facets.constructors.buckets[0].count) constructors
\(.facets.methods.buckets[0].count) methods (for example \(.facets.methodNames.buckets | map(.val) | join(", ")))
\(.facets.fields.buckets[0].count) fields (for example \(.facets.fieldNames.buckets | map(.val) | join(", ")))
\(.facets.generatedFields.buckets[0].count) new generated fields (for example \(.facets.generatedFieldNames.buckets | map(.val) | join(", ")))"'
echo DONE

## Install your own project

In [ ]:
eval $(./vars.py)
echo https://github.com/$GITHUB_ORG
echo "Create a new repo named '$SITE_NAME'"

Before you commit your vars.yaml changes, you need to create the repo in GitHub. 
Run the command below to find the URL to your GitHub organization. 

### Add vars as the initial commit to your repo

In a terminal in your own repo directory where the vars were created, you will want to initialize your GitHub repo by adding and committing your vars.yaml file and push up a new commit. Run the command below to commit and push your changes. 

In [ ]:
eval $(./vars.py)
(
  cd $SITE_SRC 
  && git init
  && git add vars.yaml
  && git commit -m 'Adding computate vars.yaml file to the project'
  && git branch -M main
  && git remote add origin $SITE_REPO
  && git push -u origin main
)
echo DONE

In [ ]:
eval $(./vars.py)
echo $SITE_REPO
echo $SITE_REPO_HTTPS